<a href="https://colab.research.google.com/github/ordevoir/Python/blob/main/20_%D0%94%D0%B2%D0%BE%D0%B8%D1%87%D0%BD%D0%BE%D0%B5_%D0%BF%D1%80%D0%B5%D0%B4%D1%81%D1%82%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B8_%D0%91%D0%B8%D1%82%D0%BE%D0%B2%D1%8B%D0%B5_%D0%BE%D0%BF%D0%B5%D1%80%D0%B0%D1%82%D0%BE%D1%80%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Двоичное представление

## Двоичная система счисления

Целое число может быть задано **двоичным целочисленным литералом** (*binary integer literal*). Например, число 12 может быть задано литералом `-0b1100`.

Целое число может быть представлено в двоичном представлении (в виде) при помощи функции `bin()`. При этом у двоичного представления будет префикс `0b`.

F-string также предоставляет возможность представления целого числа в виде строки, содержащей двоичное представление.

In [ ]:
# Двоичный целочисленный литерал
x =  0b1010  #  10
y = -0b1100  # -12

# Представление в двоичной системе счисления виде строки
bin(x), f"{y:b}", f"{y:08b}"

('0b1010', '-1100', '-0001100')

Заметим, что производится представление именно в двоичной системе, а не кодирование дополнительном коде. В ячейках памяти целые числа хранятся в дополнительном коде, поэтому нужно всегда учитывать, что последовательности нулей и единиц, возвращаемые функцией `bin()` и форматированием F-string – это представление чисел именно в двоичной системе счисления.

### Методы `bit_length()` и `bit_count()`

У объектов `int` имеются методы `bit_length()` и `bit_count()`.

Метод `bit_length()` возвращает количество бит, нужных для представления абсолютного значения числа в двоичном виде (без знака и ведущих нулей).

Метод `bit_count()` возвращает установленных битов (единиц) в двоичном представлении абсолютного значения числа.

In [ ]:
x, y = 13, 33

f"{x: b}, {y: b}"

' 1101,  100001'

In [ ]:
x.bit_length(), y.bit_length()

(4, 6)

In [ ]:
x.bit_count(), y.bit_count()

(3, 2)

## Дополнительный код

**Дополнительный код** (*two’s complement*) используется для представления целых отрицательных чисел. Положительные числа совпадают с прямым кодированием, а вот отрицательные числа представляют собой инверсию положительных, с последующим прибавлением единицы. Например число $42$ будет кодировано в `00101010`, а число $-42$ – в `11010110`. Таким образом старший бит также является знаковым. Если число представляется одним байтом, то при такой кодировке

- Имеется только один ноль: `00000000`.
- $-1$ кодируется сплошными единицами: `11111111`.
- Минимальное число $-128$ кодируется `10000000`.
- Максимальное число $127$ кодируется `01111111`.

Инверсия этого числа используется для кодирования предельного отрицательного числа ($-128$). Диапазон возможных значений $[-128, 127]$.


### Функция для отображения в дополнительном коде

Напишем функцию, которая возвращает строку, которая содержит дополнительный код числа, ограниченный слева. Для этого сформируем положительное число, двоичное представление которого соответствует дополнительному коду заданного числа:
- Выражение `1 << bits` возвращает $2^{\mathrm{bits}}$. При `bits=8` это будет число $256$ и его двоичное представление  `...0001_0000_0000`.
- Выражение `(1 << bits) - 1` возвращает значение $255$, и его двоичное представление – `...0000_1111_1111`. Таким образом мы сделали маску, которая будет обрезать слева все что левее восьмого бита при операции побитовой дизъюнкции `&`.
- В выражении `twos_zero & n` производится побитовая дизъюнкция. Оператор `&` работает непосредственно с дополнительным кодом чисел. Дополнительный код положительного числа $255$ совпадает с его двоичным представлением. Дополнительный код отрицательного числа имеет бесконечный префикс из единиц. Например, число $-12$ в дополнительном коде – `...1111_1111_0100`. Дизъюнкция с `mask` обнулит все, что левее восьмого бита, остальное останется без изменений и получится `...0000_1111_0100`.

Фактически `...0000_1111_0100` – это положительное число в дополнительном коде, равное 244. Но здесь мы получаем такое число для того, чтобы сформировать строку, содержащую дополнительный код, обрезанный слева.

In [ ]:
def to_twos(n:int, bits: int=8) -> str:
    mask = (1 << bits) - 1   # при bits=8 это будет ...000011111111
    twos_n = mask & n        # mask обнулит все, что левее 8-го бита
    return f"{twos_n:0{bits}b}"

to_twos(-1), to_twos(1, bits=16)

('11111111', '0000000000000001')

# Битовый сдвиг

**Битовый сдвиг** (*circular shift*) двоичного представления числа приводит эквивалентен умножению/делению на степени двойки.

Влево на $k$ битов (младшие нули добавляются справа):

$$
x \ll k \;=\; x · 2^k
$$

In [ ]:
x = 25
x >> 3, x // 2**3

(3, 3)


Вправо на $k$ битов (младшие разряды удаляются):
$$
x \gg k \;=\; \left\lfloor \frac{x}{2^k} \right\rfloor
$$

Сдвиг вправо приводит к делению числа на $2^k$ с округлением вниз.

In [ ]:
x = -25
x << 2, x * 2**2

(-100, -100)

Это позволяет эффективно перевести целое десятичное число из двоичное:


# Двоичное представление числа

Получить двоичное представление числа $n$ из десятичного можно путем деления на степени $2$ с округлением вниз. Степени двойки начинаются с $0$ и возрастают на единицу, образуя последовательность $1, 2, 4, 8, \dots$ Если делить $n$ на эти числа, то получим невозрастающую последовательность, которая сходится к $0$. Четные элементы этой последовательности соответствуют нулям, нечетные – единицам (в двоичном представлении числа). Первый элемент последовательности – младший бит, и далее по возрастанию, т.е. четность числа $\left\lfloor {n}/{2^k} \right\rfloor$ определяет $k$-ый бит числа. 

На практике вместо возведения в степень и целочисленного деления, эффективнее использовать битовые сдвиги: операции `n >> k` дают $\left\lfloor {n}/{2^k} \right\rfloor$. Проверку на четность можно осуществлять битовой конъюнкцией: `x & 1` возвращает $0$, если число четное, и $1$ – если нечетное. 

In [10]:
# пошаговые инструкции
import numpy as np

x = 42
ns = np.arange(7, -1, -1)
print(f"{ns = }")
shifteds = x >> ns        # результаты деления на степени двойки с округлением вниз
print(f"{shifteds = }")
odds = shifteds & 1
print(f"{odds = }")

ns = array([7, 6, 5, 4, 3, 2, 1, 0])
shifteds = array([ 0,  0,  1,  2,  5, 10, 21, 42])
odds = array([0, 0, 1, 0, 1, 0, 1, 0])


Напишем функцию, для перевода чисел в двоичное представление в виде массива.

In [12]:
import numpy as np

def int_to_bits(n: int) -> np.ndarray:
    width = n.bit_length()
    n_bytes = (width + 7) >> 3  # или // 8 
    shifts = np.arange(n_bytes*8-1, -1, -1, dtype=np.uint64)
    return (n >> shifts) & 1

In [13]:
x = 42

bynary_lst = int_to_bits(x)
print(bynary_lst)
bynary_str = "".join(bynary_lst.astype(str))
print(bynary_str)

[0 0 1 0 1 0 1 0]
00101010


# Инверсия

Инверсия целого числа $x$, закодированного в дополнительном коде, эквивалентна операции $-x - 1$. Повторное применение оператора `~`, конечно же, возвращает исходное число.

In [ ]:
x = 25
~x, ~~x

(-26, 25)

In [ ]:
to_twos(x), to_twos(~x), to_twos(~~x)   # дополнительный код

('00011001', '11100110', '00011001')

# AND, OR, XOR

Оператор конъюнкции `&` и оператор дизъюнкции `|` производят битовые операции над разрядами чисел, представленных дополнительным кодом.  Например, число $-12$, который имеет двоичное представление `1100`, в дополнительном коде с 8 разрядами будет `11110100` – т.е. производится инверсия двоичного представления положительного числа прибавляется единица.

Исключающее ИЛИ производится оператором `^`.

В Python побитовые операторы работают так, как будто числа представлены в дополнительном коде с бесконочным количеством бит. У отрицательных чисел – «бесконечные» ведущие `1`, а у положительных – `0`.

In [ ]:
x =      10  #  11
y = -0b1101  # -13

print(x, y)             # десятичное представление

10 -13


Для демонстрации того, что происходит на уровне дополнительного кода, воспользуемся функцией `to_twos()`.

## Конъюнкция битов (AND)

In [ ]:
x_twos, y_twos = to_twos(x), to_twos(y)  # дополнительный код

print(f"{x_twos}, decimal: {x}")
print(f"{y_twos}, decimal: {y}")

disjunct = x & y
disjunct_twos = to_twos(disjunct)

print(f"{disjunct_twos}, decimal: {disjunct}")


00001010, decimal: 10
11110011, decimal: -13
00000010, decimal: 2


Конъюнкция с единицей всегда возращает `1`, если второй операнд является нечетным числом, и `0`, если число четное.

In [4]:
1 & 5, 1 & 6, 1 & 0, 1 & -1

(1, 0, 0, 1)

## Дизъюнкция битов (OR)

In [ ]:
print(f"{x_twos}, decimal: {x}")
print(f"{y_twos}, decimal: {y}")

conjunct = x | y
conjunct_twos = to_twos(conjunct)

print(f"{conjunct_twos}, decimal: {conjunct}")

00001010, decimal: 10
11110011, decimal: -13
11111011, decimal: -5


## Исключающее ИЛИ (XOR)

In [ ]:
print(f"{x_twos}, decimal: {x}")
print(f"{y_twos}, decimal: {y}")

xor = x ^ y
xor_twos = to_twos(xor)

print(f"{xor_twos}, decimal: {xor}")

00001010, decimal: 10
11110011, decimal: -13
11111001, decimal: -7
